# Network analysis - Ordinary Least Squares Regression.

**Context**

This notebook loads in cell-type specifc Gene Regulatory Networks (GRNs) with the format: 'source' - 'target'. Where source refers to a Transcription Factor (TF) and target refers to a gene. The relationship i.e. row indicates that the TF under 'source' is targeting the gene under 'target' for activation of transcription. These celltype-specifc GRNs were created using the pipeline 'GRN_reconstruction_whole_data' by Abdool Al-Khaledi (a.g.al-khaledi@students.uu.nl). The pipeline takes as input a CellxGene Matrix and exports a file for each celltype containing the GRN in a 'source' - 'target' table format. This notebook carries out the subsequent part of the analysis: Modeling ASD activity as a function of total network activity and Time.

**Objective**

The purpose of this notebook is to visualize ASD and GRN activity over time. As well model ASD activity as a function of GRN activity and time to regress out these variables on ASD regulon activity.

**Summary**

The notebook begins by loading the GRN activity and ASD regulon activity. We quantify metrics for each time point including measures of GRN activity, ASD regulon activity and control TF activity. These metrics are then combined into 1 dataframe and melted into long format for the purpose of the regression analysis. 

In [ ]:
#Import packages
import pandas as pd
import numpy as np
import os
import seaborn as sns
import matplotlib.pyplot as plt
import matplotlib.patches as mpatches
import scipy.stats as stats
from scipy.stats import shapiro #test for normality
from scipy.stats import mannwhitneyu #test significance between groups
import statsmodels.formula.api as smf

In [ ]:
# Loading ASD genes 
os.chdir("/GRN_reconstruction/Mouse orthologues of SFARI genes/")

# Read csv containing gene names        
autism_genes = pd.read_csv("autism_genes.csv")

# Locate genes with SFARI gene score of 1,2 or 3        
autism_genes_1_2_3_df = pd.concat([autism_genes.loc[autism_genes['gene-score']==1], autism_genes.loc[autism_genes['gene-score']==2], autism_genes.loc[autism_genes['gene-score']==3]])

# Create gene list
autism_genes_1_2_3 = np.unique(np.unique(autism_genes_1_2_3_df['Mouse gene name']).tolist())

## Total GRN information ##

In [ ]:
# Calculate total GRN activity for each timepoint.
# Main directory
main_dir = '' # Path to directory containing a subdirectory of networks. For example: 'parent_directory/' 
              # which can contain a subfolder for each time point (E10, E11, E12...etc)
              # these subfolders should contain cell type-specific network dataframes.
    
# Create an empty dictionary to store dataframes
dfs = {}

# Iterate over each subfolder in the main directory
for subfolder in os.listdir(main_dir):

    # Input directory - update the path to point to the current subfolder
    input_dir = os.path.join(main_dir, subfolder)

    # Create directories for ASD and non-ASD if they don't exist
    ASD_output_dir = os.path.join(input_dir, 'ASD')
    non_ASD_output_dir = os.path.join(input_dir, 'non_ASD')
    if not os.path.exists(ASD_output_dir):
        os.makedirs(ASD_output_dir)
    if not os.path.exists(non_ASD_output_dir):
        os.makedirs(non_ASD_output_dir)

    # Reset lists to store results for each subfolder
    filenames = []
    GRN_activity_list = []
    unique_ASD_TFs_list = []
    free_floating_ASD_targets_list = []
    ASD_Activity_list = []
    ASD_genes_list = []
    ASD_sources_list = []

    # Your existing computations for GRN, ASD and non-ASD metrics go here, with modifications to account for subfolder-specific input and output directories
    # Iterate over files in directory
    for filename in os.listdir(input_dir):
        if filename.endswith('.csv'):
            # Read network table
            network_path = os.path.join(input_dir, filename)
            network = pd.read_csv(network_path)

            # Calculate GRN activity
            GRN_activity = network.shape[0]

            # Calculate number of unique ASD TFs in network
            unique_ASD_TFs = len(set(network['source']).intersection(autism_genes_1_2_3))

            # Calculate free-floating ASD targets
            free_floating_ASD_targets = sum(network['target'].isin(autism_genes_1_2_3) & ~network['source'].isin(autism_genes_1_2_3))

            # Calculate ASD genes count
            ASD_genes_count = network['target'].isin(autism_genes_1_2_3).sum()
            ASD_sources_count = network['source'].isin(autism_genes_1_2_3).sum()

            # Count rows where either source or target is an ASD gene "ASD edges"
            ASD_Activity = network[network['source'].isin(autism_genes_1_2_3) | network['target'].isin(autism_genes_1_2_3)].shape[0]

            # Store results in lists
            filenames.append(filename)
            GRN_activity_list.append(GRN_activity)
            unique_ASD_TFs_list.append(unique_ASD_TFs)
            free_floating_ASD_targets_list.append(free_floating_ASD_targets)
            ASD_Activity_list.append(ASD_Activity)
            ASD_genes_list.append(ASD_genes_count)
            ASD_sources_list.append(ASD_sources_count)
    # After computing the metrics and storing them in lists, create the DataFrames for each subfolder

    # GRN DataFrame
    GRN_df = pd.DataFrame({
        'Cell-type': filenames,
        'GRN_Activity': GRN_activity_list,
        'unique_ASD_TFs':unique_ASD_TFs_list,
        'free_floating_ASD_targets':free_floating_ASD_targets_list,
        'ASD_Activity':ASD_Activity_list,
        'ASD_genes': ASD_genes_list,
        'ASD_sources': ASD_sources_list
    })

    #Set index as celltype
    GRN_df.set_index('Cell-type', inplace=True)

    # Store the dataframe in the dictionary, using the subfolder name as the key
    dfs[subfolder] = GRN_df

# Now, to view the dataframes, you can do something like this:
for key in dfs:
    print(key)
    display(dfs[key])


## Create ASD directory for each time-point ##

In [ ]:
# Iterate over each 'TimePeriod' subfolder in the main directory
for time_period_folder in os.listdir(main_dir):
    # Define the input directory within the current 'TimePeriod' subfolder
    input_dir = os.path.join(main_dir, time_period_folder)

    # Define the 'ASD' output subfolder within the current 'TimePeriod' subfolder
    ASD_output_dir = os.path.join(main_dir, time_period_folder, 'ASD')

    # Create 'ASD' output subfolder if it doesn't exist
    if not os.path.exists(ASD_output_dir):
        os.makedirs(ASD_output_dir)

    # Loop over each CSV file (celltype-specifc network) in the input directory
    for filename in os.listdir(input_dir):
        if filename.endswith('.csv'):
            # Read in the network
            df = pd.read_csv(os.path.join(input_dir, filename))
            # Identify the subset of targets connected to ASD sources
            asd_nodes = set(df[df['source'].isin(autism_genes_1_2_3)]['target'])
        
            # Identify the subset of targets connected to non-ASD sources
            non_asd_nodes = set(df[~df['source'].isin(autism_genes_1_2_3)]['target'])
        
            # Identify the intersection of these target sets
            overlapping_nodes = asd_nodes.intersection(non_asd_nodes)
        
            # If an ASD source is connected to a taget in overlapping_nodes, remove that interaction
            df = df[~((df['source'].isin(autism_genes_1_2_3)) & (df['target'].isin(overlapping_nodes)))]

            # Identify the targets unique to ASD TF sources
            target_genes = set(df[df['source'].isin(autism_genes_1_2_3)]['target'])

            # Identify rows where source is ASD TF and target is unique to ASD TF(s)
            df = df[(df['source'].isin(autism_genes_1_2_3)) &  (df['target'].isin(target_genes))]

            # Export the subsetted networks to output_dire
            output_filename = os.path.join(ASD_output_dir, filename)
            df.to_csv(output_filename, index=False)


## Populate created ASD folder with the ASD regulon subset ##

In [ ]:
# Measure ASD regulon activity.

# Create a dictionary to store DataFrames for each 'TimePeriod' subfolder
ASD_GRN_dfs = {}

# Iterate over each 'TimePeriod' subfolder in the main directory
for time_period_folder in os.listdir(main_dir):
    # Define the 'ASD' subfolder within the current 'TimePeriod' subfolder
    ASD_input_dir = os.path.join(main_dir, time_period_folder, 'ASD')

    # Create empty lists to store results
    filenames = []
    ASD_GRN_activity_list = []

    # Iterate over each CSV file in the 'ASD' subfolder
    for filename in os.listdir(ASD_input_dir):
        if filename.endswith('.csv'):
            # Read in the network
            df = pd.read_csv(os.path.join(ASD_input_dir, filename))
            
            # Calculate ASD-GRN activity
            ASD_GRN_activity = df.shape[0]
        
            # Store results in lists
            filenames.append(filename)
            ASD_GRN_activity_list.append(ASD_GRN_activity)

    # Create a single dataframe 
    ASD_GRN_df = pd.DataFrame({
        'name': filenames,
        'ASD_GRN_activity': ASD_GRN_activity_list,
    })

    # Set index as celltype
    ASD_GRN_df.set_index('name', inplace=True)

    # Sort the values based on ASD_GRN_activity column
    ASD_GRN_df = ASD_GRN_df.sort_values('ASD_GRN_activity', ascending=False)
    
    # Store DataFrame in the dictionary
    ASD_GRN_dfs[time_period_folder] = ASD_GRN_df


## Create non_ASD directory in each time-point ##

In [ ]:
# Create non_ASD subsets in each subfolder 
# Iterate over each 'TimePeriod' subfolder in the main directory
for time_period_folder in os.listdir(main_dir):
    # Define the input directory within the current 'TimePeriod' subfolder
    input_dir = os.path.join(main_dir, time_period_folder)

    # Define the 'ASD' output subfolder within the current 'TimePeriod' subfolder
    non_ASD_output_dir = os.path.join(main_dir, time_period_folder, 'non_ASD')

    # Create 'ASD' output subfolder if it doesn't exist
    if not os.path.exists(non_ASD_output_dir):
        os.makedirs(non_ASD_output_dir)

    # Loop over each CSV file (celltype-specifc network) in the input directory
    for filename in os.listdir(input_dir):
        if filename.endswith('.csv'):
            # Read in the network
            df = pd.read_csv(os.path.join(input_dir, filename))
             # Identify the subset of targets connected to non-ASD sources
            non_asd_nodes = set(df[~df['source'].isin(autism_genes_1_2_3)]['target'])
        
            # Identify the subset of targets connected to ASD sources
            asd_nodes = set(df[df['source'].isin(autism_genes_1_2_3)]['target'])
        
            # Identify the intersection of these target sets
            overlapping_nodes = non_asd_nodes.intersection(asd_nodes)
        
            # If a non-ASD source is connected to a target in overlapping_nodes, remove that interaction
            df = df[~((~df['source'].isin(autism_genes_1_2_3)) & (df['target'].isin(overlapping_nodes)))]
        
            # Identify the targets unique to non-ASD TF sources
            target_genes = set(df[~df['source'].isin(autism_genes_1_2_3)]['target'])
        
            # Identify rows where source is non-ASD TF and target is unique to non-ASD TF(s)
            df = df[(~df['source'].isin(autism_genes_1_2_3)) &  (df['target'].isin(target_genes))]

            # Export the subsetted networks to output_dir
            output_filename = os.path.join(non_ASD_output_dir, filename)
            df.to_csv(output_filename, index=False)

## Populate created non_ASD folder with the ASD regulon subset ##

In [ ]:
# Create a dictionary to store DataFrames for each 'TimePeriod' subfolder
Ctrl_GRN_dfs = {}

# Iterate over each 'TimePeriod' subfolder in the main directory
for time_period_folder in os.listdir(main_dir):
    # Define the 'ASD' subfolder within the current 'TimePeriod' subfolder
    non_ASD_input_dir = os.path.join(main_dir, time_period_folder, 'non_ASD')

    # Create empty lists to store results
    filenames = []
    Ctrl_GRN_activity_list = []

    # Iterate over each CSV file in the 'ASD' subfolder
    for filename in os.listdir(non_ASD_input_dir):
        if filename.endswith('.csv'):
            # Read in the network
            df = pd.read_csv(os.path.join(non_ASD_input_dir, filename))
            
            # Calculate ctrl-GRN activity
            CT_TF_activity = df.shape[0]
        
            # Store results in lists
            filenames.append(filename)
            Ctrl_GRN_activity_list.append(CT_TF_activity)

    # Create a single dataframe 
    Ctrl_GRN_df = pd.DataFrame({
        'name': filenames,
        'Ctrl_GRN_activity': Ctrl_GRN_activity_list,
    })

    # Set index as celltype
    Ctrl_GRN_df.set_index('name', inplace=True)

    # Sort the values based on ASD_GRN_activity column
    Ctrl_GRN_df = Ctrl_GRN_df.sort_values('Ctrl_GRN_activity', ascending=False)
    
    # Store DataFrame in the dictionary
    Ctrl_GRN_dfs[time_period_folder] = Ctrl_GRN_df


## Combine the GRN, ASD regulon and control regulon information ##

In [ ]:
# New dictionary to store combined dataframes
combined_dfs = {}

# Iterate through keys of any dictionary (assuming all have the same keys)
for key in dfs.keys():
    # Join corresponding dataframes from each dictionary
    combined_dfs[key] = dfs[key].join(Ctrl_GRN_dfs[key]).join(ASD_GRN_dfs[key])



In [ ]:
# Now, to view the dataframes, you can do something like this:
for key in combined_dfs:
    print(key)
    display(combined_dfs[key])

In [ ]:
# Select 'GRN_Activity' from each DataFrame
GRN_activity_dfs = {key: df['GRN_Activity'] for key, df in combined_dfs.items()}

# Combine into a single DataFrame
GRN_activity_combined = pd.concat(GRN_activity_dfs, axis=1)

# Calculate the average across all columns
averages = GRN_activity_combined.mean(axis=1)

# Sort the dataframe based on the average values in descending order
GRN_activity_combined = GRN_activity_combined.loc[averages.sort_values(ascending=False).index]

In [ ]:
# Define the desired column order
desired_order = ['E10', 'E11',... 'P4'] # Re-order based on data.

# Reorder the DataFrame columns
GRN_activity_combined = GRN_activity_combined.reindex(columns=desired_order)


In [ ]:
GRN_activity_combined

# Visualize

In [ ]:
# Set heatmap parameters
cell_types = [os.path.splitext(os.path.basename(x))[0] for x in GRN_activity_combined.index]
sns.set(font_scale=1)
plt.figure(figsize=(18, 12))

# Draw heatmap with reduced font size for cell type labels and tick labels
sns.heatmap(GRN_activity_combined, cmap='Spectral_r', annot=False, fmt='g', linewidths=.5, 
            cbar_kws={"shrink": .5}, yticklabels=cell_types)

# Move xtick labels to top of heatmap
plt.gca().xaxis.set_ticks_position("top")

# Add x-axis label at top of graph
plt.xlabel('')
plt.gca().xaxis.set_label_position('top')
plt.gca().set_xlabel('Timepoint', fontsize=16)

# Adjust font size of cell type labels and timepoint labels
plt.tick_params(axis='both', which='major', labelsize=10)
plt.ylabel('Cell Type', fontsize=16)

# Reduce font size of cell type labels
plt.yticks(fontsize=6)

# Add color bar and title
plt.title('Cell-Type GRN activity from E7 - E18', fontsize=20)


# Save heatmap
#plt.savefig('', dpi=300)

# Show plot
plt.show()


In [ ]:
# Select 'ASD_sources' from each DataFrame
ASD_sources_activity_dfs = {key: df['ASD_sources'] for key, df in combined_dfs.items()}

# Combine into a single DataFrame
ASD_sources_activity_combined = pd.concat(ASD_sources_activity_dfs, axis=1)

# Rename columns to include timepoint
#GRN_activity_combined.columns = [f'{key} GRN_Activity' for key in GRN_activity_combined.columns]


# Calculate the average across all columns
averages = ASD_sources_activity_combined.mean(axis=1)

# Sort the dataframe based on the average values in descending order
ASD_sources_activity_combined = ASD_sources_activity_combined.loc[averages.sort_values(ascending=False).index]
# Set heatmap parameters

# Sort
import pandas as pd

# Define the desired column order
desired_order = ['E10', 'E11', 'E12', 'E13', 'E14', 'E15',
                 'E16', 'E17', 'E18', 'P1', 'P4']

# Reorder the DataFrame columns
ASD_sources_activity_combined = ASD_sources_activity_combined.reindex(columns=desired_order)


cell_types = [os.path.splitext(os.path.basename(x))[0] for x in ASD_sources_activity_combined.index]
sns.set(font_scale=1)
plt.figure(figsize=(18, 12))

# Draw heatmap with reduced font size for cell type labels and tick labels
sns.heatmap(ASD_sources_activity_combined, cmap='Spectral_r', annot=False, fmt='g', linewidths=.5, 
            cbar_kws={"shrink": .5}, yticklabels=cell_types)

# Move xtick labels to top of heatmap
plt.gca().xaxis.set_ticks_position("top")

# Add x-axis label at top of graph
plt.xlabel('')
plt.gca().xaxis.set_label_position('top')
plt.gca().set_xlabel('Timepoint', fontsize=16)

# Adjust font size of cell type labels and timepoint labels
plt.tick_params(axis='both', which='major', labelsize=10)
plt.ylabel('Cell Type', fontsize=16)

# Reduce font size of cell type labels
plt.yticks(fontsize=6)

# Add color bar and title
plt.title('Cell-Type ASD_sources_activity activity from E7 - E18', fontsize=20)


# Save heatmap
#plt.savefig('', dpi=300)

# Show plot
plt.show()


In [ ]:
# Select 'ASD_GRN_activity' from each DataFrame
ASD_GRN_activity_dfs = {key: df['ASD_GRN_activity'] for key, df in combined_dfs.items()}

# Combine into a single DataFrame
ASD_GRN_activity_combined = pd.concat(ASD_GRN_activity_dfs, axis=1)

# Rename columns to include timepoint
#GRN_activity_combined.columns = [f'{key} GRN_Activity' for key in GRN_activity_combined.columns]


# Calculate the average across all columns
averages = ASD_GRN_activity_combined.mean(axis=1)

# Sort the dataframe based on the average values in descending order
ASD_GRN_activity_combined = ASD_GRN_activity_combined.loc[averages.sort_values(ascending=False).index]
# Set heatmap parameters

# Define the desired column order
desired_order = ['E10', 'E11', 'E12', 'E13', 'E14', 'E15',
                 'E16', 'E17', 'E18', 'P1', 'P4']

# Reorder the DataFrame columns
ASD_GRN_activity_combined = ASD_GRN_activity_combined.reindex(columns=desired_order)

cell_types = [os.path.splitext(os.path.basename(x))[0] for x in ASD_GRN_activity_combined.index]
sns.set(font_scale=1)
plt.figure(figsize=(18, 12))

# Draw heatmap with reduced font size for cell type labels and tick labels
sns.heatmap(ASD_GRN_activity_combined, cmap='Spectral_r', annot=False, fmt='g', linewidths=.5, 
            cbar_kws={"shrink": .5}, yticklabels=cell_types)

# Move xtick labels to top of heatmap
plt.gca().xaxis.set_ticks_position("top")

# Add x-axis label at top of graph
plt.xlabel('')
plt.gca().xaxis.set_label_position('top')
plt.gca().set_xlabel('Timepoint', fontsize=16)

# Adjust font size of cell type labels and timepoint labels
plt.tick_params(axis='both', which='major', labelsize=10)
plt.ylabel('Cell Type', fontsize=16)

# Reduce font size of cell type labels
plt.yticks(fontsize=6)

# Add color bar and title
plt.title('Cell-Type ASD_GRN_activity activity from E7 - E18', fontsize=20)


# Save heatmap
#plt.savefig('', dpi=300)

# Show plot
plt.show()


In [ ]:
# Set the figure size
plt.figure(figsize=(12, 8))

# Create an empty list to hold the legend patches
patches = []

# Iterate over each column in the DataFrame
for column in ASD_GRN_activity_combined.columns:
    # Select the column values
    data = ASD_GRN_activity_combined[column].values.flatten()

    # Filter out the non-zero values
    data = data[data != 0]

    # Create a histogram for the column
    sns.histplot(data=data, kde=True)

    # Get the color of the KDE line (which is darker than histogram)
    line_color = plt.gca().lines[-1].get_color()

    # Add a patch to the list for the legend
    patches.append(mpatches.Patch(color=line_color, label=column))

# Set plot labels and title
plt.xlabel('ASD regulon activity', size =18)
plt.ylabel('Number of cell types', size=18)
plt.title('Distribution of ASD reuglon activities in cell types\n accross developing mouse "neo-cortex"', size =24)

# Show the legend
plt.legend(handles=patches)


# Uncomment the following line if you want to save the plot
#plt.savefig('', dpi=300)
# Show the plot
plt.show()


In [ ]:
# Select 'GRN_Activity' from each DataFrame
CTRL_GRN_activity_dfs = {key: df['Ctrl_GRN_activity'] for key, df in combined_dfs.items()}

# Combine into a single DataFrame
CTRL_GRN_activity_combined = pd.concat(CTRL_GRN_activity_dfs, axis=1)

# Rename columns to include timepoint
#GRN_activity_combined.columns = [f'{key} GRN_Activity' for key in GRN_activity_combined.columns]


# Calculate the average across all columns
averages = CTRL_GRN_activity_combined.mean(axis=1)

# Sort the dataframe based on the average values in descending order
CTRL_GRN_activity_combined = CTRL_GRN_activity_combined.loc[averages.sort_values(ascending=False).index]
# Set heatmap parameters

# Sort
# Define the desired column order
desired_order = ['E10', 'E11', 'E12', 'E13', 'E14', 'E15',
                 'E16', 'E17', 'E18', 'P1', 'P4']

# Reorder the DataFrame columns
CTRL_GRN_activity_combined = CTRL_GRN_activity_combined.reindex(columns=desired_order)


cell_types = [os.path.splitext(os.path.basename(x))[0] for x in CTRL_GRN_activity_combined.index]
sns.set(font_scale=1)
plt.figure(figsize=(18, 12))

# Draw heatmap with reduced font size for cell type labels and tick labels
sns.heatmap(CTRL_GRN_activity_combined, cmap='Spectral_r', annot=False, fmt='g', linewidths=.5, 
            cbar_kws={"shrink": .5}, yticklabels=cell_types)

# Move xtick labels to top of heatmap
plt.gca().xaxis.set_ticks_position("top")

# Add x-axis label at top of graph
plt.xlabel('')
plt.gca().xaxis.set_label_position('top')
plt.gca().set_xlabel('Timepoint', fontsize=16)

# Adjust font size of cell type labels and timepoint labels
plt.tick_params(axis='both', which='major', labelsize=10)
plt.ylabel('Cell Type', fontsize=16)

# Reduce font size of cell type labels
plt.yticks(fontsize=6)

# Add color bar and title
plt.title('Cell-Type CTRL_GRN_activity activity from E7 - E18', fontsize=20)


# Save heatmap
#plt.savefig('', dpi=300)

# Show plot
plt.show()


## OLS regression analysis

In [ ]:
# Reshape 'ASD_GRN_activity_combined' DataFrame from wide format to long format.
# 'Cell-type' is kept as identifier variable.
ASD_GRN_activity_long = ASD_GRN_activity_combined.reset_index().melt(id_vars='Cell-type', var_name='time', value_name='ASD_activity')

# Reshape 'GRN_activity_combined' DataFrame from wide format to long format.
# 'Cell-type' is kept as identifier variable.
GRN_activity_long = GRN_activity_combined.reset_index().melt(id_vars='Cell-type', var_name='time', value_name='GRN_activity')

# Merge the two reshaped dataframes on 'Cell-type' and 'time' columns.
# 'left' merge is used, so all rows from the first (left) dataframe and only the matched rows from the second (right) dataframe will be returned.
df = pd.merge(ASD_GRN_activity_long, GRN_activity_long,  how='left', left_on=['Cell-type','time'], right_on = ['Cell-type','time'])

# Define the order of the time stages
column_order = ['E10', 'E11', 'E12', 'E13', 'E14', 'E15','E16', 'E17', 'E18', 'P1', 'P4']

# Map the time stages to a numerical value for use in regression
time_dict = {col: i+1 for i, col in enumerate(column_order)}
df['time'] = df['time'].map(time_dict)

# Specify the Ordinary Least Squares (OLS) model.
# The model will regress 'ASD_activity' on 'GRN_activity' and 'time'.
model = smf.ols(formula='ASD_activity ~ GRN_activity + time', data=df)

# Fit the model using robust covariance estimation method 'HC3'.
# This method provides a robust standard errors in the case of heteroscedasticity.
results = model.fit(cov_type='HC3')

# Get the residuals from the model.
residuals = results.resid

# Printing the summary statistics of the model.
print(results.summary())

# Adding the residuals to the dataframe.
df['residuals'] = results.resid


In [ ]:
# Revert time-point dictionary
inverse_time_dict = {v: k for k, v in time_dict.items()}
df['time'] = df['time'].map(inverse_time_dict)
df

Investigate fit and residuals

In [ ]:
# Plot the histogram of residuals
plt.hist(residuals, bins='auto', density=False, alpha=0.7)
plt.xlabel('Residual value', size=12)
plt.ylabel('Number of residuals', size=12)
plt.title('Histogram of residuals', size=16)
#plt.savefig('', dpi=300)

plt.show()

# Plot the Q-Q plot of residuals
qqplot = stats.probplot(residuals, dist="norm", plot=plt)
plt.xlabel('Theoretical quantiles', size=12)
plt.ylabel('Sample quantiles', size=12)
plt.title('Q-Q plot of residuals', size=16)
#plt.savefig('', dpi=300)

plt.show()

# Perform the Shapiro-Wilk test for normality
_, p_value = stats.shapiro(residuals)
# Calculate the mean of residuals
residual_mean = residuals.mean()
print("Mean of residuals:", residual_mean)

# Set the significance level (alpha)
alpha = 0.05

# Check if the p-value is above the significance level
if p_value > alpha:
    print("Residuals are normally distributed (according to the Shapiro-Wilk test).")
else:
    print("Residuals are not normally distributed (according to the Shapiro-Wilk test).")



In [ ]:
# Add residuals to original DataFrame
df['residuals'] = residuals

# Sort the DataFrame by residuals
df_sorted_by_residuals = df.sort_values(by='residuals')


from IPython.display import display, HTML

# Enable scrolling
pd.set_option('display.max_rows', None)

# Convert DataFrame to HTML and set a CSS property to make the table scrollable
df_html = df_sorted_by_residuals.to_html()
styled_df_html = '<style> .dataframe {height: 400px; overflow-y: auto;}</style>' + df_html

# Display HTML
display(HTML(styled_df_html))


Visualize ASD regulon activity as a function of time

In [ ]:
# Get the predicted values
df['predicted_ASD_activity'] = results.fittedvalues

# Create a plot
plt.figure(figsize=(10, 6))

# Plot actual ASD activity
sns.lineplot(x=df['time'], y=df['ASD_activity'], label='Actual ASD regulon activity')

# Plot predicted ASD activity (GRN activity held constant)
sns.lineplot(x=df['time'], y=df['predicted_ASD_activity'], label='Predicted ASD regulon activity')

plt.title('Predicted Vs. Actual average ASD regulon activity\n over mouse neo-cortex development', size =22)
plt.xlabel('Time', size=12)
plt.ylabel('ASD regulon activity', size=12)
plt.legend()
# Move legend to the left of the plot
plt.legend(loc='center left', bbox_to_anchor=(0., 0.7))
plt.savefig('', dpi=300)

plt.show()


In [ ]:
# Create a plot
plt.figure(figsize=(10, 6))

# Plot actual ASD activity
plt.scatter(df['time'], df['ASD_activity'], label='Actual ASD regulon activity')

# Plot predicted ASD activity
plt.scatter(df['time'], df['predicted_ASD_activity'], label='Predicted ASD regulon activity')

plt.title('Predicted Vs. Actual ASD regulon activity\n over mouse neo-cortex development', size =24)
plt.xlabel('Time', size=16)
plt.ylabel('ASD regulon activity', size=16)
plt.legend()

# Uncomment the following line if you want to save the plot
plt.savefig('', dpi=300)

plt.show()


## END 